## Chapter 7: Functional Programming

### 7.1: Functional vs. Non-functional forms

In [1]:
arr=collect(1:5)

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

In [4]:
newarr=zeros(Int,5) # this an array of zeros of length 5 
for i=1:5
  newarr[i]=arr[i]^2 
end
newarr

5-element Array{Int64,1}:
  1
  4
  9
 16
 25

In [5]:
f(x)=x^2
arr=[1,2,3,4,5]
map(f,arr)

5-element Array{Int64,1}:
  1
  4
  9
 16
 25

In [6]:
map(x->x^2,arr)

5-element Array{Int64,1}:
  1
  4
  9
 16
 25

### 7.2: Reducing an array

In [8]:
A=[1,2,3,4,5]

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

In [9]:
reduce((x,y)-> x+y, A)

15

In [10]:
reduce((x,y)->x*y,A)

120

In [11]:
numPos(arr) = reduce((n,val) -> val > 0 ? n+1 : n, arr, init=0)

numPos (generic function with 1 method)

In [12]:
numPos([-3,5,8,-2,11])

3

### 7.3: The mapreduce function

In [13]:
mapreduce(x->x^2,+,[1,2,3],init=0)

14

In [14]:
sum(x->x^2,[1,2,3])

14

### 7.4: Mapping a Function over an 2D array

In [15]:
A=[i+j for i=1:10,j=1:3]

10×3 Array{Int64,2}:
  2   3   4
  3   4   5
  4   5   6
  5   6   7
  6   7   8
  7   8   9
  8   9  10
  9  10  11
 10  11  12
 11  12  13

In [16]:
mapslices(sum,A,dims=[1])

1×3 Array{Int64,2}:
 65  75  85

In [17]:
mapslices(sum,A,dims=[2])

10×1 Array{Int64,2}:
  9
 12
 15
 18
 21
 24
 27
 30
 33
 36

### 7.5: Writing Fast Code

In [18]:
function sum1(n::Int)
  local arr = collect(1:n)
  local sum = 0
  for i in arr
    sum += i
  end
  sum
end

sum1 (generic function with 1 method)

In [19]:
@time sum1(1_000_000)

  0.004541 seconds (2 allocations: 7.629 MiB)


500000500000

In [20]:
@time sum1(1_000_000_000)

 23.782032 seconds (2 allocations: 7.451 GiB, 0.04% gc time)


500000000500000000

In [21]:
function sum2(n::Integer)
    local sum = 0
    for i=1:n
        sum+=i
    end
    sum
end

sum2 (generic function with 1 method)

In [22]:
@time sum2(1_000_000_000)

  0.000000 seconds


500000000500000000

In [24]:
@time sum2(100_000_000_000)

  0.000000 seconds


932356074711512064

In [25]:
function sum3(n::Int)
    local sum = big(0)
    for i=1:n
        sum+=i
    end
    sum
end

sum3 (generic function with 1 method)

In [26]:
@time sum3(1_000_000)

  0.995349 seconds (3.00 M allocations: 45.776 MiB, 77.16% gc time)


500000500000

In [27]:
@time sum3(10_000_000)

  2.304307 seconds (30.00 M allocations: 457.764 MiB, 26.37% gc time)


50000005000000

In [28]:
function sum4(n::Int) 
  reduce(+,1:big(n))
end

sum4 (generic function with 1 method)

In [29]:
@time sum4(1_000_000)

  0.325424 seconds (5.00 M allocations: 91.553 MiB, 17.02% gc time)


500000500000

In [30]:
@time sum4(10_000_000)

  3.749989 seconds (50.00 M allocations: 915.528 MiB, 18.87% gc time)


50000005000000

In [31]:
@time sum(1:big(10)^6)

  0.000287 seconds (50 allocations: 1008 bytes)


500000500000

In [32]:
@time sum(1:big(10)^20)

  0.000012 seconds (50 allocations: 1.070 KiB)


5000000000000000000050000000000000000000

### 7.7: Computing Fibonacci Numbers

In [38]:
function fibonacci(n::Integer)
  n==1 || n==2 ? 1 : fibonacci(n-1) + fibonacci(n-2)
end

fibonacci (generic function with 1 method)

In [39]:
map(fibonacci,1:10)

10-element Array{Int64,1}:
  1
  1
  2
  3
  5
  8
 13
 21
 34
 55

In [40]:
@time fibonacci(40)

  0.344543 seconds


102334155

In [41]:
@time fibonacci(41)

  0.540270 seconds


165580141

In [24]:
num_evals = 0

0

In [44]:
function fibonacciEval(n::Integer) 
  global num_evals
  if n==1 || n==2
    num_evals +=1
    return 1 
  else
    num_evals += 2
    return fibonacciEval(n-1) + fibonacciEval(n-2)
  end
end

fibonacciEval (generic function with 1 method)

In [45]:
num_evals=0
fibonacciEval(5)
num_evals

13

In [46]:
num_evals=0
fibonacciEval(20)
num_evals

20293

In [48]:
num_evals=0
fibonacciEval(21)
num_evals

32836

In [51]:
function fibonacci2(n::Integer)
  numbers = zeros(Int64,n)  # need an array of length n
  numbers[1:2] .= 1         # the first two fibonacci numbers are 1
  function fib(k::Integer)
    if numbers[k] > 0
      return numbers[k]  # if we know the number just return it
    end
    numbers[k-1] = fib(k-1)  # these two lines call fib! recursively, but 
    numbers[k-2] = fib(k-2)  # if the value is known return it
    numbers[k] = numbers[k-1] + numbers[k-2]  # still the same definition.
  end
  fib(n)
end

fibonacci2 (generic function with 1 method)

In [54]:
@time fibonacci2(100)

  0.000018 seconds (197 allocations: 3.953 KiB)


3736710778780434371

In [39]:
@time fibonacci3(100)

  0.000013 seconds (197 allocations: 3.953 KiB)


3736710778780434371

In [84]:
function fibonacci3(n)
  local x,y = (1,1)
  for i = 1:n-1
    x,y = (y, x+y)
  end
  x
end

fibonacci3 (generic function with 1 method)

In [86]:
@time fibonacci3(3)

  0.000000 seconds


2